In [3]:
import numpy as np
import bokeh as bk
from bokeh.io import push_notebook
import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.io import push_notebook
from ipywidgets import interact
import ipywidgets as widgets
bkp.output_notebook()

Loading BokehJS ...

# Analytical model for spherical sludge growth in Al-Fe system

## References:

1. An Analytical Solidification Model for Solute Redistribution during of Planar, Columnar, or Equiaxed Morphology L. Nastac and D.M. Stefanescu: Metall. Trans. A, 1993, vol. 24A, pp. 2107-18

2. Computational Modeling of NbC/Laves Formation In INCONEL 718 Equiaxed Castings L. Nastac and D.M. Stefanescu: Metall. Mater. Trans. A, 1997, vol. 28A, pp. 1582-1587

# Introduction:

## Goal: 

Application of Nastac-Stefanescu model for morphology of sludge ($ \beta $ aluminum) particles

## Assumptions:

<ul>
    <li>Instantaneous nucleation</li>
    <li>Sludge growth only in liquid</li>
    <li>negligible interference between growing
sludge</li>
    <li>volume diffusion limited growth of sludge.</li>
    <li>Constant growth velocity</li>
    
</ul>

## Schematic:

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/22/nucleus.png" /img>

<strong> Figure 1. Schematic of volume element (from ref.1 ) </strong>

# Calculation strategy:

<ol>
    <li> Determine the material properties of Al </li>
    <li> Determine the material properties of Fe in Al </li>
    <li> Heat transfer: Nova </li>
    <li> Determine final solidification time </li>
    <li> Experimentally determine final sludge particle radius as function of cooling rate </li>
    <li> calculate interface growth velocity: </li>
    <li> $$ V_{\beta} = \frac{\Omega_{Fe}}{\Phi_{\beta}} \frac{D_L}{R_D-R_f} $$ </li>
    <li> $$ \Phi_{\beta} = \frac{4 \pi R_{\beta}}{A_s} $$ </li>
    <li> $$ R_D = \frac{3}{4\pi} \Bigg( \frac{v_l}{N_{\beta}} \Bigg)^{1/3} $$ </li>
    <li> $$ \Omega_{Fe} = \frac{C_L^* - C_0}{C_L^*-C_S^*} $$ </li>
    <li> Calculate interface radius: </li>
    <li> $$ R^* = V_{\beta}t $$ </li>
    <li> calculate f raction of solid: </li>
    <li> $$ f_s=(R^*/Rf)^3 $$</li>
    <li> Calculate interface liquid iron concentration as function of fraction of solid: </li>
    <li> $$ C_L^*= \frac{C_0\big( kI_S +I_L -\frac{1}{3} \big) + Q }{kI_S + I_L - \frac{1}{3} + \frac{1-k f_S}{3}} $$</li>
    <li> $$ Q= \frac{v_{\beta}}{v_{\Omega}}\frac{\rho_{\beta}}{\rho_{eu}} pct \; Fe_\beta  $$ </li>
    <li> $$ I_S = \frac{2f_S}{\pi^2} \sum_{n=1}^{\infty} \frac{1}{n^2} \Bigg[-\big( \frac{n \pi}{f_s^{1/3}} \big) \frac{D_St}{R^2_f} \Bigg]  $$ </li>
    <li> $$ I_L = 2f_s^{2/3}(1-f_s^{1/3}) \sum_{n=1}^{\infty} \frac{1}{\alpha^2_n} exp \Bigg[ -\Big( \frac{\alpha_n}{1/f_s^{1/3}} \Big)^2 \frac{D_L t}{R^2_f} \Bigg] $$ </li>
    <li> $$ \alpha_n R_f  \cot {\alpha_n} = R_f - R^* $$ </li>
    <li> $$ \langle C^L \rangle =  $$ </li>
    <li> Calculate sludge content (wt%) as function of fraction of solid </li>
</ol>

# Implementation:

## Experimental sludge size:

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/24/sludge.png" />

# Heat transfer model (0D):

    1. A 383 as Al-Si system with Si-equivalent 
    2. Use constant heat flow:

Heat flow:

$$ \dot{Q} = \rho \Delta H_f \; \frac{d f_s}{d t} - \rho c  \frac{dT}{dt} $$

$$ T^{new} = T^{old} - \frac{\dot{Q}}{\rho c} \Delta t + \frac{\Delta H_f}{c} \Delta f_s^{new} $$

For N spherical dendrites in volume element:

$$ \frac{\partial f_s}{\partial t} = \frac{\partial}{\partial t} \Big( N \frac{4}{3} \pi r^3 \Big) = 4 \pi r^2 N\frac{\partial r}{\partial t}  $$

$$ \frac{\partial r}{\partial t} = V_S $$

Therefore:

$$ r_s^{new} = r_s^{old} + V_s^{new} \Delta t $$

$$ \Delta f_s^{new} =  4 \pi (r_s^{new})^2 N V_S^{new} \Delta t $$

$$ f_s^{new} = f_s^{old} + \Delta f_s^{new} $$

Solidification velocity, assuming solutal dendrite:

$$ V_S^{new} = \mu \Delta T^2 = \frac{D_L}{2\cdot \pi^2 \Gamma m(k-1) \langle C_L^{new} \rangle} \big(\Delta T^{new} \big)^2 $$

Scheil for liquid concentration:

$$ \langle C_L^{new} \rangle = C_0 (1-f_S^{old})^{k-1} $$

Undercooling:

$$ \Delta T = \Delta T_C + \Delta T_T = m(C_0 - C_L) + T^* - T_{bulk} = T_f + mC_0 - T_{bulk} = T_f + m\langle C_L^{new} \rangle - T_{bulk} $$

Data taken from Nova database:

<table>
    <tr>
        <th>Material property</th>
        <td>$ \rho_s $</td>
        <td> $ c_s $ </td>
        <td> $ k_s $ </td>
        <td> $ \Delta H_{FP} $ </td>
        <td> $ \Delta H_{FE} $ </td>
        <td> $  T_L $ </td>
        <td> $  T_S $ </td>
        <td> $ \Delta T_0 $ </td>
        <td> $ T_E $ </td>
        <td> $ C_0 $ </td>
        <td> $ m $ </td>
        <td> $ k $ </td>
        <td> $ \Gamma $ </td>
        <td> $ D_L $ </td>
        <td> $ C_E $ </td>
    </tr>
    <tr>
        <th>value</th>
        <td> 2485 </td>
        <td> 964.3 </td>
        <td> 96 </td>
        <td> 340 000 </td>
        <td> 508 497 </td>
        <td> 867 K </td>
        <td> 812 K </td>
        <td> 55 K </td>
        <td> 803 K </td>
        <td> 10.1 </td>
        <td> -14.56 </td>
        <td> 0.1539 </td>
        <td>  1.96e-07  </td>
        <td> 5.4e-09 </td>
        <td> 14.5 </td>
    </tr>
</table>

In [536]:
#general
q=2*10**(8)
temp_init=903.15
delta_t=0.01

#primary
Gamma=1.96*10**(-7)
C_0=10
D_L=5.4*10**(-9)
k=0.1539
N=100
rho=2485
c=964.3
delta_H_f=3.4*10**5
T_L = 867
m=-14
C_E=14.5
T_E=803

#eutectic
mu_eut_alpha = 2.9*10**(-6)
mu_eut_beta = 5.8*10**(-5)
delta_H_f_eut = 508497


def C_L(fs_old,C_L_old):
    if C_L_old < C_E:
        return C_0*(1-fs_old)**(k-1)
    else:
        return C_L_old

def Delta_T(C_L_new,T_old):
    if C_L_new < C_E:
        if T_L+m*(C_L_new-C_0)-T_old >= 0:
            return T_L+m*(C_L_new-C_0)-T_old
        else:
            return 0
    else:
        return T_E-T_old
        
def V_S(C_L_new,Delta_T_new):
    if C_L_new < C_E:
        return D_L/(2*np.pi**2*Gamma*m*(k-1)*C_L_new)*(Delta_T_new)**2
    else:
        return mu_eut_alpha*(Delta_T_new)**2
                
def R(r_old,V_S_new):
    return r_old+V_S_new*delta_t
                
def Delta_fs(r_new,V_S_new):
    return 4*np.pi*r_new**(2)*V_S_new*delta_t*N
    
def T(T_old,Delta_fs_new,C_L_new):
    if C_L_new < C_E:
        return T_old-q/(rho*c)*delta_t+delta_H_f/c*Delta_fs_new
    else:
        return T_old-q/(rho*c)*delta_t+delta_H_f_eut/c*Delta_fs_new

def fs(fs_old,Delta_fs_new):
    return fs_old+Delta_fs_new

max_steps=1000

time=np.linspace(0,delta_t*max_steps,max_steps)
fs_arr=np.zeros(max_steps)

T_arr=np.zeros(max_steps)
T_arr[0]=temp_init

r_arr=np.zeros(max_steps)
r_arr[0]=10**(-7)

C_L_arr=np.zeros(max_steps)
C_L_arr[0]=C_0

V_S_arr=np.zeros(max_steps)
V_S_arr[0]=0
                
Delta_T_arr=np.zeros(max_steps)
                
Delta_fs_arr=np.zeros(max_steps)

n=0

for i in range(1,len(time)):
    C_L_arr[i] = C_L(fs_arr[i-1],C_L_arr[i-1])
    Delta_T_arr[i] = Delta_T(C_L_arr[i],T_arr[i-1])
    V_S_arr[i] = V_S(C_L_arr[i],Delta_T_arr[i])
    r_arr[i] = R(r_arr[i-1], V_S_arr[i])
    Delta_fs_arr[i] = Delta_fs(r_arr[i],V_S_arr[i]) 
    fs_arr[i] = fs(fs_arr[i-1],Delta_fs_arr[i])
    T_arr[i]= T(T_arr[i-1],Delta_fs_arr[i],C_L_arr[i])
    if fs_arr[i] >= 1:
        n=i
        break
        
print(n)

738


In [540]:
f=bkp.figure(title="Evolution of T during dendrite growth", x_axis_label="Time,s", y_axis_label="Temperature, K")

f.line(time[0:n],T_arr[0:n], color="red", legend="Temperature")

bkp.show(f)

In [538]:
f=bkp.figure(title="Evolution of fs dendrite growth", x_axis_label="Time,s", y_axis_label="fs")


f.line(time[0:n],fs_arr[0:n], color="red", legend="fraction of solid")

bkp.show(f)

In [539]:
f=bkp.figure(title="Liquid concentration", x_axis_label="Time,s", y_axis_label="C_L")

f.line(time[0:n],C_L_arr[0:n], color="red", legend="Liquid concentration")

bkp.show(f)

## Cooling rate:

In [50]:
#Cooling rate, K/S
cooling_rate=np.array([1.2,1.7,2.8,10,25])
print(cooling_rate)

[  1.2   1.7   2.8  10.   25. ]


## Rf:

In [9]:
#Sludge area, um^2
sludge_area=np.array([380,290,240,235,245])
#Sludge area, m^2
sludge_area=sludge_area*10**(-12)
#Sludge radius, m
sludge_radius=np.sqrt(sludge_area)
print(sludge_radius)

[  1.94935887e-05   1.70293864e-05   1.54919334e-05   1.53297097e-05
   1.56524758e-05]


## Intermetallic growth:

In [25]:
cr=cooling_rate[0]
R_f=sludge_radius[0]
t_f=20
C_0=10.127
k=0.1539
N_beta=5
v_l=3/4*np.pi*R_f**3
D_L=2.69*10**(-9)
delta_t=1
pct_Fe_beta=18.15
rho_beta=2300
rho_eut=2300

R_D=(3/(4*np.pi))*(v_l/N_beta)**(1/3)


def sphere_volume(R):
    return (3/4*np.pi*R**3)

def sol_sup_sat(C_l):
    return (C_l-C_0)/(C_l-C_l/k)
    
def phi(R_beta,A_beta):
    return (4*np.pi*R_beta/A_beta)

def V_calc(C_l,R_beta,A_beta):
    return sol_sup_sat(C_l)/phi(R_beta,A_beta)*D_L/(R_D-R_beta)

def Q(R_beta):
    return sphere_colume(R_beta)/v_l*(rho_beta/rho_eut)

def I_S():

C_l=np.zeros(t_f)
V_beta=np.zeros(t_f)
R_beta=np.zeros(t_f)
A_beta=np.zeros(t_f)
f_s=np.zeros(t_f)

C_l[0]=C_0


V_beta[0]=V_calc(C_l[0],R_beta[0],A_beta[0])

R_beta[1]=R_beta[0]+V_beta[0]*delta_t
f_s[1]=(R_beta[1]/R_f)**3






    

for i in range(0,t_f,1):
    break
    
print(C_l_star)
    

IndentationError: expected an indented block (<ipython-input-25-bcbb1d9cfaa0>, line 34)